In [1]:
# coding: utf-8

# In[ ]:

from __future__ import print_function

import sys, os
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")
    
    # Overlay
    ol = Overlay("FFT_iFFT.bit")
    ipFFT  = ol.FFT_0  
    ipiFFT = ol.iFFT_0

    # Memory allocate
    inBufferFFT  = allocate(shape=(1024,), dtype=np.float64)
    inBufferiFFT = allocate(shape=(1024,), dtype=np.float64)
        
    # ============= Set Kernel Argument ==============
    # control
    # 0x00 : Control signals
    #        bit 0  - ap_start (Read/Write/COH)
    #        bit 1  - ap_done (Read/COR)
    #        bit 2  - ap_idle (Read)
    # 0x10 : Data signal of f
    #        bit 31~0 - f[31:0] (Read/Write)
    # 0x14 : Data signal of f
    #        bit 31~0 - f[63:32] (Read/Write)
    # 0x1c : Data signal of logn
    #        bit 31~0 - logn[31:0] (Read/Write)
    logn = 10
    ipFFT.write(0x1C, logn) # Coefficient
    ipiFFT.write(0x1C, logn)
    ipFFT.write(0x10, inBufferFFT.device_address) # 0x10 bit 31:0
    ipiFFT.write(0x10, inBufferiFFT.device_address) # 0x10 bit 31:0

    # ================ Data Transfer =================
    with open("FFT_in.txt", "r+") as FFT_in:
        for i in range(1024):
            line = FFT_in.readline()
            inBufferFFT[i] = float(line)

    timeKernelStart = time()
    ipFFT.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
    while (ipFFT.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("FFT execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
    
    for i in range(1024):
        inBufferiFFT[i] = inBufferFFT[i]
    timeKernelStart = time()
    ipiFFT.write(0x00, 0x01)
    while (ipiFFT.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("iFFT execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
    # ================== Compare =====================
    error_cnt = 0
    
    with open("FFT_in.txt" , "r") as result:
        for i in range(1024):
            line = result.readline()
            golden = float(line)

            if round(inBufferiFFT[i], 5) != golden:
                print(f"Failed, expect value: {golden:.17f}")
                error_cnt += 1
            #else:
            #   print(f"Calculate value: {inBufferiFFT[i]:.17f}")

    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")

    print("============================")
    print("Exit process")



Entry: /usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py"
FFT execution time: 0.002037525177001953 s
iFFT execution time: 0.0034952163696289062 s
Test Passed!!
Exit process
